## Historical Max Temperature from Tomorrow.io API 

We retrieve the max temperature (i.e., `temperatureMax`) for Delhi, India on [H3 resolution 7](https://h3geo.org) from the [Historical](https://docs.tomorrow.io/reference/historical) data layer of the [Tomorrow.io API](https://docs.tomorrow.io/reference/api-introduction).

In [1]:
%reload_ext autoreload
%autoreload 2

import asyncio
import json
import os
from typing import Optional

import geopandas
import h3
import pandas as pd
import shapely
from aiohttp import ClientSession
from shapely.geometry import Polygon

from utilities import weatherFunctions

INFO:utilities:Initializing custom packages


The following are auxiliary function(s). Ideally, these will be packaged in an opportune moment.

In [2]:
def get_h3_tessellation(
    gdf: geopandas.GeoDataFrame, name="shapeName", resolution=10
) -> geopandas.GeoDataFrame:
    mapper = dict()
    tiles = set()

    # TODO: vectorize, if possible
    for idx, row in gdf.iterrows():
        geometry = row["geometry"] 
        match geometry.geom_type:
            case "Polygon":
                hex_ids = h3.polyfill(
                    shapely.geometry.mapping(geometry),
                    resolution,
                    geo_json_conformant=True,
                )

                tiles = tiles.union(set(hex_ids))
                mapper.update([(hex_id, row[name]) for hex_id in hex_ids])

            case "MultiPolygon":
                for x in geometry.geoms:
                    hex_ids = h3.polyfill(
                        shapely.geometry.mapping(x),
                        resolution,
                        geo_json_conformant=True,
                    )

                    tiles = tiles.union(set(hex_ids))
                    mapper.update([(hex_id, row[name]) for hex_id in hex_ids])
            case _:
                raise (Exception)

    tessellation = geopandas.GeoDataFrame(
        data=tiles,
        geometry=[Polygon(h3.h3_to_geo_boundary(idx, True)) for idx in tiles],
        columns=["hex_id"],
        crs="EPSG:4326",
    )

    return tessellation

In [3]:
# create the Tomorrow.io API token at https://app.tomorrow.io/development/keys
TOMORROW_API_KEY = <KEY>

## Area of Interest: Chennai

In [51]:
# PHILIPPINES  = geopandas.read_file('../../data/shapefiles/philippines/phl_adminboundaries_candidate_exclude_adm3/phl_admbnda_adm2_psa_namria_20200529.shp')
# MANILA = PHILIPPINES[PHILIPPINES['ADM2_EN'].isin(['NCR, City of Manila, First District', 'NCR, Second District', 'NCR, Third District', 'NCR, Fourth District'])]
#PHILIPPINES_ADM3 = geopandas.read_file('../../data/shapefiles/philippines/phl_adminboundaries_candidate_adm3/phl_admbnda_adm3_psa_namria_20200529.shp')
#Manila_Neighbors = PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin(['Rodriguez', 'San Mateo', 'City of San Pedro', 'Obando'])]

#PHILIPPINES_ADM3 = geopandas.read_file('../../data/shapefiles/philippines/phl_adminboundaries_candidate_adm3/phl_admbnda_adm3_psa_namria_20200529.shp')
ZAMBOANGA = PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin(['Zamboanga City'])]

# SAN_MATEO=PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin([])]
# SAN_PEDRO=PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin([])]
# OBANDO=PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin([])]
#MANILA2 = PHILIPPINES[PHILIPPINES['ADM2_EN'].isin(['NCR, Second District'])]
#MANILA3 = PHILIPPINES[PHILIPPINES['ADM2_EN'].isin(['NCR, Third District'])]
#MANILA4 = PHILIPPINES[PHILIPPINES['ADM2_EN'].isin(['NCR, Fourth District'])]


# DAVAO = PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin(['Davao City'])]
# ZAMBOANGA = PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin(['Zamboanga City'])]
# CEBU = PHILIPPINES_ADM3[PHILIPPINES_ADM3['ADM3_EN'].isin(['Cebu City'])]

## Tessellation in H3

In this step, we generate a tessellation layer for the **area of interest** using [H3 resolution 7](https://h3geo.org).

In [52]:
TESSELLATION = get_h3_tessellation(ZAMBOANGA, name='ADM3_EN', resolution=7)

In [53]:
TESSELLATION.explore()

In [54]:
TESSELLATION['centre'] = TESSELLATION['geometry'].apply(lambda x: x.centroid)

In [55]:
TESSELLATION["geojson"] = TESSELLATION["geometry"].apply(
    lambda x: json.dumps(shapely.geometry.mapping(x))
)

TESSELLATION["centre_geojson"] = TESSELLATION["centre"].apply(
    lambda x: json.dumps(shapely.geometry.mapping(x))
)

## Retrieve `Historical` from Tomorrow.io API

> Tomorrow.io's Historical Weather API allows you to query weather conditions (limited to historical data layers ]) by specifying the location (GeoJSON of a Point, LineString or Polygon), fields ("temperature", "windSpeed", ...), timesteps ("1h", "1d") and the startTime and endTime, such that the response include a historical timeline.

    Polygon and Polyline limits:

    Polygon - 10,000 square km and no more than 70km per segment.
    Polyline - 2,000 km long.
    Max number of vertices - 550.
    Timerange is limited to up to 30 days per API call.

    If the location is a Polygon or a Polyline, you can specify whether you want the min/max/avg values throughout that coverage area by adding them as a suffix to any of the available fields (temperatureMax, temperatureMaxTime) and if not specified the response will default to Max.

    The historical archive is based on a reanalysis model that blends past short-range weather forecasts with observations through advanced data assimilation techniques. The historical archive data deviates from the recent historical data -7 days since it is based on a different observation data assimilation system that incorporates a larger set of final observational records.
    Please note that our reanalysis model takes between 7 to 90 days to calculate the data fields. Please see the historical data field Availability.

    See also: https://docs.tomorrow.io/reference/historical-overview

In [56]:
class TomorrowAPIClient:
    """An Asynchronous API client for Tomorrow.io API"

    Parameters
    ----------
    token : str
        Tomorrow.io API token

    Notes
    -----
    For more information, please see https://docs.tomorrow.io
    """

    BASE_URL = "https://api.tomorrow.io/v4"

    def __init__(
        self, session: Optional[ClientSession] = None, token: Optional[str] = None
    ):
        self.session = session or ClientSession()
        self.semaphore = asyncio.BoundedSemaphore(4)
        self.token = token or os.getenv("TOMORROW_TOKEN")

    async def __aenter__(self):
        return self

    async def __aexit__(self, *args):
        await self.close()

    async def close(self):
        await self.session.close()

    async def post(self, url, json, params={}, headers={}):
        params["apikey"] = self.token
        async with self.semaphore, self.session.post(
            url, json=json, params=params, headers=headers
        ) as response:
            return await response.json()

### Creating `intervals`

Let's start in 2021, from January 1st to December 31th. The Tomorrow.io API limits the date range to 30 days. Thus, we create 13 periods of 28 days and add 1 day to the last period.

In [58]:
date_range = pd.date_range("2023-04-01 12:00:00", "2023-04-30 12:00:00", periods=30)
intervals = list(zip(date_range, date_range[1:]))

In [59]:
intervals

[(Timestamp('2023-04-01 12:00:00'), Timestamp('2023-04-02 12:00:00')),
 (Timestamp('2023-04-02 12:00:00'), Timestamp('2023-04-03 12:00:00')),
 (Timestamp('2023-04-03 12:00:00'), Timestamp('2023-04-04 12:00:00')),
 (Timestamp('2023-04-04 12:00:00'), Timestamp('2023-04-05 12:00:00')),
 (Timestamp('2023-04-05 12:00:00'), Timestamp('2023-04-06 12:00:00')),
 (Timestamp('2023-04-06 12:00:00'), Timestamp('2023-04-07 12:00:00')),
 (Timestamp('2023-04-07 12:00:00'), Timestamp('2023-04-08 12:00:00')),
 (Timestamp('2023-04-08 12:00:00'), Timestamp('2023-04-09 12:00:00')),
 (Timestamp('2023-04-09 12:00:00'), Timestamp('2023-04-10 12:00:00')),
 (Timestamp('2023-04-10 12:00:00'), Timestamp('2023-04-11 12:00:00')),
 (Timestamp('2023-04-11 12:00:00'), Timestamp('2023-04-12 12:00:00')),
 (Timestamp('2023-04-12 12:00:00'), Timestamp('2023-04-13 12:00:00')),
 (Timestamp('2023-04-13 12:00:00'), Timestamp('2023-04-14 12:00:00')),
 (Timestamp('2023-04-14 12:00:00'), Timestamp('2023-04-15 12:00:00')),
 (Time

Fix by adding last day, 

### Create `payloads`

In [60]:
payloads = [
    {
        "location": location,
        "fields": ["temperatureMin", 'humidityMin'],
        "timesteps": ["1d"],
        "startTime": startTime.isoformat(),
        "endTime": endTime.isoformat(),
        "units": "metric",
    }
    for location in TESSELLATION["geojson"]
    for (startTime, endTime) in intervals
]

In [61]:
len(payloads)

7975

Just checking the cardinality,

In [62]:
len(payloads) == len(TESSELLATION) * len(intervals)

True

In [17]:
payloads

[{'location': '{"type": "Polygon", "coordinates": [[[121.02348215151316, 14.436437723433103], [121.02588986118877, 14.42383751951722], [121.03813526771623, 14.420163571446368], [121.04797426844507, 14.429091146114983], [121.04556640603099, 14.441692561869724], [121.03331969545023, 14.445365191213625], [121.02348215151316, 14.436437723433103]]]}',
  'fields': ['temperatureMin', 'humidityMin'],
  'timesteps': ['1d'],
  'startTime': '2023-04-01T12:00:00',
  'endTime': '2023-04-02T12:00:00',
  'units': 'metric'},
 {'location': '{"type": "Polygon", "coordinates": [[[121.02348215151316, 14.436437723433103], [121.02588986118877, 14.42383751951722], [121.03813526771623, 14.420163571446368], [121.04797426844507, 14.429091146114983], [121.04556640603099, 14.441692561869724], [121.03331969545023, 14.445365191213625], [121.02348215151316, 14.436437723433103]]]}',
  'fields': ['temperatureMin', 'humidityMin'],
  'timesteps': ['1d'],
  'startTime': '2023-04-02T12:00:00',
  'endTime': '2023-04-03T12:

Now, let's call the Tomorrow.io API!

In [63]:
async with TomorrowAPIClient(token=TOMORROW_API_KEY) as client:

    url = f"https://api.tomorrow.io/v4/historical"
    headers = {"Accept": "application/json", "Content-Type": "application/json"}

    futures = [client.post(url, json=payload, headers=headers) for payload in payloads]
    zamboanga = await asyncio.gather(*futures)

In [20]:
def get_values(x, field):
    if x is None:
        return None
    elif len(x[0]['intervals'])==0:
        return 0
    elif len(x[0]['intervals'])==1:
        #print(x[0]['intervals'])
        return x[0]['intervals'][0]['values'][field]
    else: 
        return np.mean([x[0]['intervals'][0]['values'][field], x[0]['intervals'][1]['values'][field]])

In [23]:
import numpy as np
def data_manipulation(manila, humidity, temperature):
    df = pd.json_normalize(manila).reset_index().merge(TESSELLATION.merge(pd.json_normalize(payloads), left_on = 'geojson', right_on ='location').reset_index())
    df = df[~df['data.timelines'].isna()]
    df[humidity] = df['data.timelines'].apply(lambda x: get_values(x, humidity))
    df[temperature] = df['data.timelines'].apply(lambda x: get_values(x, temperature))
    df = df[['geometry', 'startTime', 'endTime', humidity, temperature, 'hex_id', 'centre_geojson']]

    ftemp = temperature+'_F'

    df[ftemp] = df[temperature].apply(lambda x: weatherFunctions.convert_celcius_to_fahrenheit(x))
    
    # Make a change to use this convert function from the weatherFunctions module
    df['heat_index'] = df.apply(lambda x: weatherFunctions.calculate_heat_index(x[ftemp], x[humidity]), axis=1)
    df['heat_index_C'] = df['heat_index'].apply(lambda x: weatherFunctions.convert_fahrenheit_to_celcius(x))

    return df

In [64]:
zamboanga_heatindex_min= data_manipulation(zamboanga, 'humidityMin', 'temperatureMin')

In [65]:
zamboanga_heatindex_min.to_csv('../../data/tomorrow.io/zamboanga_april2023_min_heat_index.csv')

In [167]:
from utilities.weatherFunctions import *

In [192]:
df.to_csv('../../data/tomorrow.io/manila_20230420_20230427.csv')